In [73]:
#imports
import os
import reverse_geocoder as rg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter

icd = 'SOLAR_PV_UTILITY_TEXAS.xlsx' #all 254 counties in TX

non_ercot_counties = ['El Paso','Hudspeth','Gaines','Terry','Yoakum','Cochran','Hockley','Lubbock','Bailey',
                      'Lamb','Hartley','Dallam','Moore','Sherman','Hansford','Hutchinson','Ochiltree','Lipscomb', 
                      'Hemphill','Bowie','Morris','Cass','Camp','Marion','Upshur','Gregg','Harrison','Panola',
                      'Shelby','San Augustine','Sabine','Trinity','Polk','Tyler','Jasper','Newton','San Jacinto',
                      'Hardin','Liberty','Orange','Jefferson']

removec = [count + ' County' for count in non_ercot_counties]

cdat = pd.read_excel(icd, usecols=['County'])
counties = cdat['County'].to_list()

for count in removec:
    counties.remove(count)

print(len(counties))

213


I have stored all data NREL Wind Toolkit and SAM Solar data in google drive for space considerations.
Pydrive is being employed to access and work with this data - next 3 blocks are for doing this.
Data is converted to 'Wind2009.csv' type files, which are used in modeling and analysis.

In [2]:
#pydrive quickstart google authorization
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

#pydrive drive interaction
from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=62179872653-sp0h6kju465i2lb634qqqqtlkogst5fk.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [ ]:
#solar reformatting

#get files from drive
file_list = drive.ListFile({'q': "'' in parents and trashed=false"}).GetList()
print(len(file_list))
#2009 - 1kG13cMZ0XdGwO3lA1x_2C7QlWIj2SKUH
#2010 - 1WkUW03_DWJqj5OgYpNe9bsijrtUwQzJc
#2011 - 1pqhBKFg6FGNOqTwyNs9dq5J2XChaWBUh

#reformatting - want average capacity of all coordinates within county
countydic = dict.fromkeys(counties,np.zeros(17520)) #every half hour
countdic = dict.fromkeys(counties,0)
wcn = 0 #how many points in our mesh grid are not within an ERCOT county

for file in file_list:
    name = file['title']
    lat = name.split('lat')[1]
    [lat,lon] = lat.split('lon')
    lon = lon.split('.csv')[0]
    lon = lon.split(' ')[0]
    coordinates = (float(lat),float(lon))
    county = rg.search(coordinates)[0]['admin2'] #reverse_geocoder used to site coordinates in county
    print(county)
    if county in counties:
        file = drive.CreateFile({'id': file['id']})
        file.GetContentFile('file.csv')
        pdat = pd.read_csv('file.csv', usecols=['PowerGen_kW'],dtype=np.float).to_numpy()
        pdat = [float(i) for i in pdat]
        countydic[county] = countydic[county] + pdat
        countdic[county] += 1
    else:
        wcn += 1

nameplate_cap = 22557.15 #nameplate capacity of used sample sites in kW
outdic = dict.fromkeys(counties,np.zeros(17520))
for county in counties:
    if countdic[county] != 0:
        #computing average solar capacity within each county
        outdic[county] = countydic[county] / (countdic[county]*nameplate_cap)

df = pd.DataFrame(data=outdic).to_csv('Solar.csv')

print(wcn)

In [ ]:
#wind reformatting

#get files from drive
file_list = drive.ListFile({'q': "'1H40oHl84pa8iaJ7xY4UpUlY78kvMlc2l' in parents and trashed=false"}).GetList()
#reformatting - if you want max site per county
countydic = {}
sumdic = defaultdict(int) #keeping track of best site so far (in terms of average capacity factor)
#filedic = {} #keeping track of best location within each county - not necesary
wcn = 0 #how many points in our mesh grid are not within an ERCOT county

#if you want average per county
countydic = dict.fromkeys(counties,np.zeros(17520)) #every half hour
countdic = dict.fromkeys(counties,0)

for file in file_list:
    name = file['title']
    lat = name.split('lat')[1]
    [lat,lon] = lat.split('lon')
    lon = lon.split('.csv')[0]
    coordinates = (float(lat),float(lon))
    county = rg.search(coordinates)[0]['admin2']
    if county in counties:
        file = drive.CreateFile({'id': file['id']})
        file.GetContentFile('file.csv')
        pdat = pd.read_csv('file.csv', skiprows=3, usecols=['Capacity Factor'],dtype=np.float, engine='python').to_numpy()
        dat = []
        ssum = 0
        for hour in range(17520):
            power = sum([float(i) for i in pdat[(6*hour):(6*(hour+1))]]) / 6 #5 min --> 30 min intervals
            dat.append(power)
            ssum += power
        #for getting max site
#         if ssum > sumdic[county]: #is this site "better" than current best site in county
#             countydic[county] = dat
#             sumdic[county] = ssum
            #filedic[county] = [lat,lon]
            #capdic[county] = scap
        
        #for getting average site
        countydic[county] = countydic[county] + dat
        countdic[county] += 1    
    else:
        wcn += 1

#for getting average site, need to take average
outdic = dict.fromkeys(counties,np.zeros(17520))
for county in counties:
    if countdic[county] != 0:
        outdic[county] = countydic[county] / (countdic[county])

pd.DataFrame(data=outdic).to_csv('Wind.csv')
print(wcn)

Loading formatted geocoded file...


In [ ]:
#import energy price data by the half hour for each load region
#not using HB_HOUSTON (houston hub), COAST --> HB_SOUTH, NORTH --> HB_WEST
y = 17520
loadzones = ['HB_NORTH', 'HB_SOUTH', 'HB_WEST']
loadprices = dict.fromkeys(loadzones,np.empty(0))

epsheet = 'EnergyPricesERCOT2019.xlsx'
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
xls = pd.ExcelFile(epsheet)

for i in range(12):
    df = pd.read_excel(xls, months[i], usecols=['Settlement Point Name','Repeated Hour Flag','Settlement Point Price'])
    df = df.loc[df['Repeated Hour Flag']=='N']
    for zone in loadzones:
        eprice = df.loc[df['Settlement Point Name']==zone]['Settlement Point Price'].to_numpy()
        eprice = (eprice[::2] + eprice[1::2])/2
        loadprices[zone] = np.hstack((loadprices[zone],eprice))

#godam daylight savings time
for zone in loadzones:
    eprice = loadprices[zone]
    eprice = np.concatenate((eprice[0:3268],np.array([eprice[3267],eprice[3268]]),eprice[3268:17518]))
    loadprices[zone] = eprice

pd.DataFrame(data=loadprices).to_csv('EnergyPriceNew2019.csv')

In [ ]:
#FOR RUNNING ACTUAL OPTIMIZATION

In [26]:
#REGIONS (if u want)

from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

pdat = pd.read_excel('Zipcode_Data.xlsx', sheet_name='ZipToZone', usecols=['Svc. Address ZIP Code','Weather Zone Code'])

#removing zipcodes outside of ERCOT, one's with discrespancies just keeping cuz they dont even know wuts right
update = pd.read_excel('Zipcode_Data_Update.xlsx', sheet_name='D1', skiprows=3, usecols=['Zipcode']).to_numpy().flatten()
dropidx = pdat.index[pdat['Svc. Address ZIP Code'].isin(update)].to_list()

pdat = pdat.drop(dropidx)

regions = np.array(['NORTH', 'NCENT', 'EAST', 'COAST', 'SOUTH', 'SCENT', 'WEST', 'FWEST']) #easier if numpy array
regdict = {'NORTH': [], 'NCENT': [], 'EAST': [], 'COAST': [], 'SOUTH': [], 'SCENT': [], 'WEST': [], 'FWEST': []}

for i,row in pdat.iterrows():
    zipcode = str(row['Svc. Address ZIP Code'])
    region = str(row['Weather Zone Code'])
    county = search.by_zipcode(zipcode).to_dict()['county']
    regdict[region].append(county)
    
#could put multiple region counties in primary region or just leave in multiple regions
#every county pretty dominantly in one county except austin (even split)
simpregdict = {}
multregdict = {}
for county in counties:
    counter = []
    for reg in regions:
        counter.append(regdict[reg].count(county))
    simpregion = regions[np.argmax(counter)]
    multregions = list(regions[[i for i, val in enumerate(counter) if val != 0]])
    simpregdict[county] = simpregion #for simple can do county --> region cause only 1 region per county
    multregdict[county] = multregions

simpregdict['Austin County'] = 'SCENT' #austin between two zones put in SCENT b/c map looks like that

In [66]:
#months (if u want)
days = np.array([31,28,31,30,31,30,31,31,30,31,30,31]); halfhours = days*48; hsums = []
for month in range(12):
    hsums.append(sum(halfhours[:month]))
hsums.append(17520)
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

In [55]:
#IF DOING FREE MODEL (all counties available)

solarcounts = counties
solarcnum = len(solarcounts)
solarcaps = np.ones(solarcnum)
solarnames = [solarcounts[i] + ' Solar: ' for i in range(solarcnum)]

windcounts = counties
windcnum = len(windcounts)
windcaps = np.ones(windcnum)
windnames = [windcounts[i] + ' Wind: ' for i in range(solarcnum)]


In [29]:
#IF USING GIS REPORT (SKIP if doing free model)

#import GIS site data
gisreport = 'Ercot_GIS_Report_June.xlsx'
pdat = pd.read_excel(gisreport, sheet_name='Project Details', usecols=['GINR Study Phase','County','Fuel','Capacity (MW)'])

#only care about sites which already have IA agreement (for now)
names = ["SS Completed, FIS Completed, IA", "SS Completed, FIS Started, IA","SS Completed, FIS Completed, No IA",
        "SS Completed, FIS Started, No IA"]
#pdat = pdat.loc[pdat['GINR Study Phase'].isin(names)]

solarp = pdat.loc[pdat['Fuel']=="SOL"]
solarp = solarp.loc[solarp['Capacity (MW)'] > 0] #some caps 0 - don't need to remove technically but y not
#solarp = solarp.loc[simpregdict[solarp['County'] + ' County'] == 'SCENT'] #for jkspruce model

ogsolarcounts = solarp['County'].to_list()
solarcounts = [count + ' County' for count in ogsolarcounts]
solarcaps = solarp['Capacity (MW)'].to_numpy()
solarcnum = len(solarcounts)

windp = pdat.loc[pdat['Fuel']=="WIN"]
windp = windp.loc[windp['Capacity (MW)'] > 0] 
#windp = windp.loc[simpregdict[windp['County'] + ' County'] == 'SCENT'] #for jkspruce model

ogwindcounts = windp['County'].to_list()
windcounts = [count + ' County' for count in ogwindcounts]
windcaps = windp['Capacity (MW)'].to_numpy()
windcnum = len(windcounts)

solarnames = [solarcounts[i] + ' Solar: ' + str(solarcaps[i]) for i in range(solarcnum)]
windnames = [windcounts[i] + ' Wind: ' + str(windcaps[i]) for i in range(windcnum)] 

####for checking if things ran correctly
print(solarcnum)
print(windcnum)

print(sum(solarcaps))
print(sum(windcaps))
print(max(solarcaps))
print(max(windcaps))


262
108
58014.529999999984
24588.49
610.0
630.0


In [56]:
#import formatted power data and scale up by site capacity, put in matrix form
solarcsvs = ['NewSolar2009.csv','NewSolar2010.csv','NewSolar2011.csv']
solartraincsvs = solarcsvs
solartestcsvs = []
solarpower = np.empty((solarcnum,0))
solarpowertest = np.empty((solarcnum,0))
hl = 1 #working with half hours so hour-length = 0.5

for solarcsv in solartraincsvs:
    pdat = pd.read_csv(solarcsv, usecols=solarcounts)
    solaryear = pdat[solarcounts[0]].to_numpy() * solarcaps[0] * hl #0th
    for i in range(solarcnum-1): #1th onwards
        solaryear = np.vstack((solaryear,pdat[solarcounts[i+1]].to_numpy() * solarcaps[i+1] * hl))
    solarpower = np.concatenate((solarpower,solaryear),axis=1) #div by 2 to convert from MW (power) to MWh (energy output)

for solarcsv in solartestcsvs:
    pdat = pd.read_csv(solarcsv, usecols=solarcounts)
    solaryear = pdat[solarcounts[0]].to_numpy() * solarcaps[0] * hl #0th
    for i in range(solarcnum-1): #1th onwards
        solaryear = np.vstack((solaryear,pdat[solarcounts[i+1]].to_numpy() * solarcaps[i+1] * hl))
    solarpowertest = np.concatenate((solarpowertest,solaryear),axis=1)

windcsvs = ['CountyWind2009.csv','Wind2010.csv','Wind2011.csv']
windcsvsavg = ['WindAvg2009.csv','WindAvg2010.csv','WindAvg2011.csv']
windtraincsvs = windcsvs
windtestcsvs = []
windpower = np.empty((windcnum,0))
windpowertest = np.empty((windcnum,0))

for windcsv in windtraincsvs:
    pdat = pd.read_csv(windcsv, usecols=windcounts)
    windyear = pdat[windcounts[0]].to_numpy() * windcaps[0] * hl #0th
    for i in range(windcnum-1): #1th onwards
        windyear = np.vstack((windyear,pdat[windcounts[i+1]].to_numpy() * windcaps[i+1] * hl))
    windpower = np.concatenate((windpower,windyear),axis=1)
    
for windcsv in windtestcsvs:
    pdat = pd.read_csv(windcsv, usecols=windcounts)
    windyear = pdat[windcounts[0]].to_numpy() * windcaps[0] * hl #0th
    for i in range(windcnum-1): #1th onwards
        windyear = np.vstack((windyear,pdat[windcounts[i+1]].to_numpy() * windcaps[i+1] * hl))
    windpowertest = np.concatenate((windpowertest,windyear),axis=1)
    
solarpower = np.transpose(solarpower)
windpower = np.transpose(windpower)
solarpowertest = np.transpose(solarpowertest)
windpowertest = np.transpose(windpowertest)

print(solarpower.shape)

(52560, 213)


In [60]:
sp = sum(solarpower) / 52560
wp = sum(windpower) / 52560
outdic = {'County': counties, 'Solar': sp, 'Wind': wp}
print(counties[1:10])
#pd.DataFrame(data=outdic).to_csv('Fig6Data.csv')

['Andrews County', 'Angelina County', 'Aransas County', 'Archer County', 'Armstrong County', 'Atascosa County', 'Austin County', 'Bandera County', 'Bastrop County']


In [27]:
print(dsum[windcounts.index('Pecos County')])
print(dsum[windcounts.index('Willacy County')])

0.06252267035806358
0.052028138497158184


In [4]:
#import coal data and format to be by the half hour

fuel = 'Fuel2019.xlsx'
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
xls = pd.ExcelFile(fuel)

coalhours = np.empty(0)

for i in range(12):
    dat = pd.read_excel(xls, months[i])
    coal = dat.loc[dat['Fuel']=="Coal"]
    coal = coal[coal.columns[4::2]].to_numpy() + coal[coal.columns[5::2]].to_numpy() #every half hour
    coal = np.reshape(coal,np.prod(coal.shape))
    coalhours = np.concatenate((coalhours,coal))
coalhours = np.nan_to_num(coalhours)

#coal = 'Coal2019.csv'
#coalhours = pd.read_csv(coal,usecols=['Coal']).to_numpy()

coalhours = np.hstack((coalhours,coalhours,coalhours))


In [33]:
outdic = {}
for m in range(12):
    avg = np.empty(48)
    month = coalhours[hsums[m]:hsums[m+1]]
    for hh in range(48):
        avg[hh] = sum(month[hh::48]) / days[m]
    outdic[months[m]] = avg
pd.DataFrame(data=outdic).to_csv('CoalSupplement.csv')

In [ ]:
#import coal plant data and format to be regional
coalcsv = 'EPA_Coal_2019.csv'
pdat = pd.read_csv(coalcsv,usecols=['Facility_Name','Gross_Load_MW'])

oklaunion = pdat.loc[pdat['Facility_Name']=="Oklaunion Power Station"]['Gross_Load_MW'].to_numpy()
coletocreek = pdat.loc[pdat['Facility_Name']=="Coleto Creek"]['Gross_Load_MW'].to_numpy()
martinlake = pdat.loc[pdat['Facility_Name']=="Martin Lake"]['Gross_Load_MW'].to_numpy()
fayette = pdat.loc[pdat['Facility_Name']=="Sam Seymour"]['Gross_Load_MW'].to_numpy()
sanmiguel = pdat.loc[pdat['Facility_Name']=="San Miguel"]['Gross_Load_MW'].to_numpy()
sandycreek = pdat.loc[pdat['Facility_Name']=="Sandy Creek Energy Station"]['Gross_Load_MW'].to_numpy()
oakgrove = pdat.loc[pdat['Facility_Name']=="Oak Grove"]['Gross_Load_MW'].to_numpy()
jkspruce = pdat.loc[pdat['Facility_Name']=="J K Spruce"]['Gross_Load_MW'].to_numpy()
limestone = pdat.loc[pdat['Facility_Name']=="Limestone"]['Gross_Load_MW'].to_numpy()
waparish = pdat.loc[pdat['Facility_Name']=="W A Parish"]['Gross_Load_MW'].to_numpy()
twinoaks = pdat.loc[pdat['Facility_Name']=="Twin Oaks"]['Gross_Load_MW'].to_numpy()
#welsh = pdat.loc[pdat['Facility_Name']=="Welsh Power Plant"]['Gross_Load_MW'].to_numpy()

#costs = [0, 120.2, 102.2, 84.35, 126.34, 0, 79.02, 89.53, 102.48, 163.99, 0, 116.6]
    
southcoal = coletocreek + sanmiguel
scentcoal = jkspruce + fayette
ncentcoal = limestone + sandycreek
northcoal = oklaunion
eastcoal = martinlake + oakgrove + twinoaks
coastcoal = waparish
westcoal = np.zeros(8760)
fwestcoal = np.zeros(8760)
allcoalplants = southcoal + scentcoal + ncentcoal + northcoal + eastcoal + coastcoal + westcoal + fwestcoal

coalload = {'HB_NORTH': ncentcoal + eastcoal, 'HB_SOUTH': southcoal + scentcoal + coastcoal, 'HB_WEST': westcoal + fwestcoal + northcoal}
coalregs = {'NORTH': northcoal, 'NCENT': ncentcoal, 'EAST': eastcoal, 'COAST': coastcoal, 'SOUTH': southcoal, 'SCENT': scentcoal, 'WEST': westcoal, 'FWEST': fwestcoal, 'All': allcoalplants}
coalregscf = {}
for reg, regcoal in coalregs.items():
#    coalregs[reg] = np.hstack((regcoal,regcoal,regcoal))
    m = max(regcoal)
    if m < 0.1:
        coalregscf[reg] = regcoal
    else:
        coalregscf[reg] = regcoal / max(regcoal)

#San Miguel in Atascosa county which is 9/10 zipcodes in SOUTH (and 1 in SCENT) --> putting in south

In [ ]:
loadzones = ['HB_NORTH', 'HB_SOUTH', 'HB_WEST']
weathertoload = {'NORTH': 'HB_WEST', 'NCENT': 'HB_NORTH', 'EAST': 'HB_NORTH', 'COAST': 'HB_SOUTH', 'SOUTH': 'HB_SOUTH', 'SCENT': 'HB_SOUTH', 'WEST': 'HB_WEST', 'FWEST': 'HB_WEST'}
energyprice = pd.read_csv('EnergyPriceAgg.csv')

In [13]:
#import cost data
sc = 92.50*1000 # LCOE $ / MW-Cap 
solarcost = sc*solarcaps
wc = 118.48*1000 # LCOE $ / MW-Cap 
windcost = wc*windcaps
coalcost = 111.67 # $ / MWh

In [14]:
#Exploratory analysis - 2300 infeasible hours and 5 wind cites with 0 capacity! (to go along w/ the negative ones)
coaltotal = sum(coalhours)
h = len(coalhours)

spv = np.sum(solarpower, axis=1); wpv = np.sum(windpower, axis=1);
allpower = spv + wpv
feasible = [i for i in range(h) if allpower[i] >= coalhours[i]]
feasn = len(feasible)

#slackbound = 0.01*coalhours #percent of hour demand I allow to be slacked
maxcap = 1 #set to linear max for linear case
infeasibility = 0
infhours = 0
badhours = []
si = 0
wi = 0
for i in range(h):
    dif = coalhours[i] - maxcap*(allpower[i])
    sd = coalhours[i] - spv[i]
    wd = coalhours[i] - wpv[i]
    if dif > 0:
        infeasibility += dif
        infhours += 1
        badhours.append(i)
        #slackbound[i] += dif
    if sd > 0:
        si += sd
    if wd > 0:
        wi += wd
        
slacknames = ['Slack:' + str(i) for i in range(h)] # also need this for naming zeta (hour) variables
xsnames = ['Excess:' + str(i) for i in range(h)]

#slowbound = 1
#wlowbound = 1
#energy_price = 50

print(infeasibility/coaltotal)
# print(si / coaltotal)
# print(wi / coaltotal)
print(infhours/h)
print(h / (48*365))
print(max(coalhours))

print((sum(solarcost) + sum(windcost)) / 10**9)
print((sum(solarcaps) + sum(windcaps)) / 10**3)

0.043012797958400684
0.12203196347031964
3.0
6949.897196
8.2795883202
82.60301999999999


In [15]:
#setting up model and solving
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("coal")

#vars
s = m.addMVar(solarcnum, lb=0,ub=1, vtype=GRB.BINARY, name=solarnames) #indicator var for if site is built
w = m.addMVar(windcnum, lb=0,ub=1, vtype=GRB.BINARY, name=windnames)
slack = m.addMVar(h, lb=0, ub=6950, vtype=GRB.CONTINUOUS, name=slacknames)
#power = m.addMVar(h, lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name=xsnames)
#hours = m.addMVar(h, lb=0, ub=1, vtype=GRB.BINARY, name=hournames[0:h])

#for continuous model
#s = m.addMVar(solarcnum, lb=0,ub=610, vtype=GRB.CONTINUOUS, name=solarnames)
#w = m.addMVar(windcnum, lb=0,ub=630, vtype=GRB.CONTINUOUS, name=windnames)

#objective
m.setObjective((solarcost @ s) + (windcost @ w), GRB.MINIMIZE)

#constraints
for hour in range(h):
    m.addConstr((solarpower[hour,:] @ s) + (windpower[hour,:] @ w) >= coalhours[hour] - slack[hour])
    #m.addConstr((solarpower[hour,:] @ s) + (windpower[hour,:] @ w) >= coalhours[hour] - hours[hour]*6950)
m.addConstr(sum(slack) <= 0.1*coaltotal) #allow % of total coal demand to be slacked
#m.addConstr(sum(hours) >= 0.7*17520) #if you want to meet certain percent of hours

#this is to make linear sites be above certain threshold (ur not gonna make tiny sites)
#for site in range(solarcnum):
#    m.addConstr(or_(s[site] == 0,s[site] >= slowbound),"orconstr")
#for site in range(windcnum):
#    m.addConstr(or_(w[site] == 0,w[site] >= wlowbound),"orconstr")


#optimizingc
#m.Params.MIPGap = 10**(-6)
m.optimize()


Using license file /Users/richard.morse/gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-22
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 52561 rows, 52930 columns and 19487667 nonzeros
Model fingerprint: 0x0729fb43
Variable types: 52560 continuous, 370 integer (370 binary)
Coefficient statistics:
  Matrix range     [3e-06, 3e+02]
  Objective range  [2e+05, 7e+07]
  Bounds range     [1e+00, 7e+03]
  RHS range        [1e+03, 2e+07]
Found heuristic solution: objective 8.132089e+09
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 0 rows and 28 columns (presolve time = 45s) ...
Presolve removed 0 rows and 28 columns (presolve time = 46s) ...
Presolve removed 6395 rows and 6423 columns (presolve time = 60s) ...
Presolve removed 6395 rows and 6423 columns (presolve time = 60s) ...
Presolve removed 6395 rows and 6425 columns (pres

     0     0 3.0217e+09    0    3 3.0227e+09 3.0217e+09  0.03%     -  687s
     0     0 3.0217e+09    0    3 3.0227e+09 3.0217e+09  0.03%     -  688s
     0     0 3.0217e+09    0    3 3.0227e+09 3.0217e+09  0.03%     -  689s
     0     0 3.0217e+09    0    3 3.0227e+09 3.0217e+09  0.03%     -  691s
     0     0 3.0217e+09    0    3 3.0227e+09 3.0217e+09  0.03%     -  692s
     0     0 3.0217e+09    0    5 3.0227e+09 3.0217e+09  0.03%     -  693s
     0     0 3.0217e+09    0    5 3.0227e+09 3.0217e+09  0.03%     -  695s
     0     0 3.0217e+09    0    5 3.0227e+09 3.0217e+09  0.03%     -  696s
     0     0 3.0217e+09    0    5 3.0227e+09 3.0217e+09  0.03%     -  697s
     0     0 3.0217e+09    0    5 3.0227e+09 3.0217e+09  0.03%     -  698s
     0     0 3.0217e+09    0    5 3.0227e+09 3.0217e+09  0.03%     -  699s
     0     0 3.0217e+09    0    5 3.0227e+09 3.0217e+09  0.03%     -  700s
     0     0 3.0217e+09    0    5 3.0227e+09 3.0217e+09  0.03%     -  702s
     0     0 3.0217e+09  

In [18]:
#investigating model and reporting results
#h = len(coalhours)
slackedhours = 0 #how many hours did we not meet (significantly i.e. more than 1% off)
slacksum = 0 #how much total load did we not meet
powersum = 0
slackout = []
solarout = []
windout = []
smallsites = 0

for v in m.getVars():
    if v.x > 0.01:
        name = v.varName
        if 'Wind' in name:
            windout.append(name)
            if v.x < 1:
                smallsites += 1
            
        elif 'Solar' in name:
            solarout.append(name)
            if v.x < 1:
                smallsites += 1
        elif 'Slack' in name:
            slackedhours += 1
            slacksum += v.x
            [nada,hour] = name.split(':')
            slackout.append(int(hour))
        else:
            powersum += v.x

sint = [round(val) for val in s.x] 
wint = [round(val) for val in w.x]
sitecost = np.matmul(solarcost,sint) + np.matmul(windcost,wint)
#coaltotalcost = sum(slack.x)*coalcost
solarpowerout = np.matmul(solarpower,sint)
windpowerout = np.matmul(windpower,wint)
powerout = solarpowerout + windpowerout
#energyrevenue = np.matmul(energyprice['HB_HUBAVG'],powerout)
#profit = energyrevenue - sitecost
    
print('Solar sites used: %g' % len(solarout))
print('Wind sites used: %g' % len(windout))
print('Percent of Hours slacked: %g' % (slackedhours/h))
print('Percent of Load slacked: %g' % (slacksum / coaltotal))
#print('Obj (Billion): %g' % (m.objVal  / 10**9))
print('Build Cost (Billion) %g' % (sitecost / 10**9))
#print('Revenue (Billion) %g' % (energyrevenue / 10**9))

#saving site output
#pd.DataFrame(data={'Solar': solarpowerout, 'Wind': windpowerout}).to_csv('PowerGIS.csv')

#saving solver info
#m.write('out.mst')
#m.write('out.attr')
#m.write('out.mps')

Solar sites used: 44
Wind sites used: 79
Percent of Hours slacked: 0.322983
Percent of Load slacked: 0.0999992
Obj (Billion): 3.02188
Build Cost (Billion) 3.02188


In [ ]:
mc = (sitecost*3) / (sum(powerout))
print(mc)
pdat = pd.read_csv('Power90.csv')
modelpower = pdat['Solar'].to_numpy() + pdat['Wind'].to_numpy()
dif = modelpower - coalhours
surplus = 0; slack = 0;
for d in dif:
    if d > 0:
        surplus += d
    else:
        slack += abs(d)
print(surplus / (3*10**3),slack / (3*10**3))

In [ ]:
solarmodel = defaultdict(float)
windmodel = defaultdict(float)
for site in solarout:
    [county,cap] = site.split(' Solar: ')
    solarmodel[county] += float(cap)
for site in windout:
    [county,cap] = site.split(' Wind: ')
    windmodel[county] += float(cap)
    
#model power output save (all 3 years - each half hour - of solar and wind output seperatley
pd.DataFrame(data={'Solar': solarpowerout, 'Wind': windpowerout}).to_csv('PowerOut.csv')
pd.DataFrame(data={'Slack': slack.x}).to_csv('SlackOut.csv')

pd.DataFrame(data=solarmodel,index=['Capacities']).to_csv('SolarOutSites.csv')
pd.DataFrame(data=windmodel,index=['Capacities']).to_csv('WindOutSites.csv')
pd.DataFrame(data={'Sites': solarout}).to_csv('SitesS.csv')
pd.DataFrame(data={'Sites': windout}).to_csv('SitesW.csv')

In [ ]:
#Distribution
coalcap = 14225
#coal = pd.read_csv('Coal2019.csv',usecols=['Coal']).to_numpy().flatten()
#coal = np.flip(np.sort(2*coal))
#coalcf = coal / coalcap

pdat = pd.read_csv('Power90.csv')
solarp = 2*pdat['Solar'].to_numpy()
windp = 2*pdat['Wind'].to_numpy()
power = solarp + windp

# y = 17520
# syear1 = solarp[0:y]
# syear2 = solarp[y:2*y]
# syear3 = solarp[2*y:3*y]
# wyear1 = windp[0:y]
# wyear2 = windp[y:2*y]
# wyear3 = windp[2*y:3*y]
# ayear1 = power[0:y]
# ayear2 = power[y:2*y]
# ayear3 = power[2*y:3*y]

solarcap = 11706.08
windcap = 16611.35
totalcap = solarcap + windcap

# spd1 = np.flip(np.sort(syear1 / spoweravg))
# spd2 = np.flip(np.sort(syear2 / spoweravg))
# spd3 = np.flip(np.sort(syear3 / spoweravg))
# wpd1 = np.flip(np.sort(wyear1 / wpoweravg))
# wpd2 = np.flip(np.sort(wyear2 / wpoweravg))
# wpd3 = np.flip(np.sort(wyear3 / wpoweravg))
# apd1 = np.flip(np.sort(ayear1 / apoweravg))
# apd2 = np.flip(np.sort(ayear2 / apoweravg))
# apd3 = np.flip(np.sort(ayear3 / apoweravg))

solarp = np.flip(np.sort(solarp / solarcap))
windp = np.flip(np.sort(windp / windcap))
power = np.flip(np.sort(power / totalcap))

#yeardic = {'Solar 2009': spd1, 'Solar 2010': spd2, 'Solar 2011': spd3, 'Wind 2009': wpd1, 'Wind 2010': wpd2, 'Wind 2011': wpd3, 'Both 2009': apd1, 'Both 2010': apd2, 'Both 2011': apd3}
alldic = {'Solar': solarp, 'Wind': windp, 'Both': power}

#pd.DataFrame(data={'Coal Dist': coalyear}).to_csv('CoalDistrYear.csv')
#pd.DataFrame(data={'Coal Dist': coalall}).to_csv('CoalDistrExtended.csv')
#pd.DataFrame(data=yeardic).to_csv('ModelDistrYears.csv')
pd.DataFrame(data=alldic).to_csv('ModelDistr.csv')


In [ ]:
#Monthly average Capacity Factor graph for each fuel type
solarpoweragg = np.sum(solarpower,axis=1) #/ 213 #213 counties
windpoweragg = np.sum(windpower,axis=1) #/ 213

coal = 'Coal2019.csv'
coalhours = pd.read_csv(coal,usecols=['Coal']).to_numpy()

solaryear = np.empty(17520)
windyear = np.empty(17520)
for i in range(17520):
    solaryear[i] = sum(solarpoweragg[i::17520])/3
    windyear[i] = sum(windpoweragg[i::17520])/3

solaryear = 2*solaryear / sum(solarcaps)
windyear = 2*windyear / sum(windcaps)
coalhours = 2*coalhours / 15065
days = [31,28,31,30,31,30,31,31,30,31,30,31]
d=0

sma = np.empty(12); wma = np.empty(12); cma = np.empty(12);
for month in range(12):
        sma[month] = sum(solaryear[(d*48):((d+days[month])*48)]) / (days[month]*48)
        wma[month] = sum(windyear[(d*48):((d+days[month])*48)]) / (days[month]*48)
        cma[month] = sum(coalhours[(d*48):((d+days[month])*48)]) / (days[month]*48) #coal max = 6950
        d += days[month]

'''
#same graph for each region
smregs = {}
wmregs = {}
cmregs = {}

for reg in regions:
    sm = np.empty(12)
    wm = np.empty(12)
    cm = np.empty(12)
    d = 0
    
    solar3year = solarregcf[reg]
    wind3year = windregcf[reg]
    solaryear = np.empty(17520)
    windyear = np.empty(17520)
    for i in range(17520):
        solaryear[i] = sum(solar3year[i::17520])/3
        windyear[i] = sum(wind3year[i::17520])/3
    coalregpower = coalregscf[reg]
    
    for month in range(12):
        sm[month] = sum(solaryear[(d*48):((d+days[month])*48)]) / (days[month]*48)
        wm[month] = sum(windyear[(d*48):((d+days[month])*48)]) / (days[month]*48)
        cm[month] = sum(coalregpower[(d*24):((d+days[month])*24)]) / (days[month]*24*1) #coal max = 6950
        d += days[month]
    smregs[reg] = sm
    wmregs[reg] = wm
    cmregs[reg] = cm
''' 
smregs = {'Solar': sma}
wmregs = {'Wind': wma}
cmregs = {'Coal': cma}
pd.DataFrame(data=smregs).to_csv('SolarMonthlyCF.csv')
pd.DataFrame(data=wmregs).to_csv('WindMonthlyCF.csv')
pd.DataFrame(data=cmregs).to_csv('CoalMonthlyCF.csv')

In [ ]:
y = 17520; a=0*y;b=1*y;
pdat = pd.read_csv('PowerGIS.csv')
gispower = pdat['Solar'].to_numpy() + pdat['Wind'].to_numpy()
gispower = 2*gispower[a:b]
pdat = pd.read_csv('Power90.csv')
modelpower = pdat['Solar'].to_numpy() + pdat['Wind'].to_numpy()
modelpower = 2*modelpower[a:b]
pdat = pd.read_csv('Coal2019.csv')
coalpower = 2*pdat['Coal'].to_numpy()

gisday = np.empty(48)
modelday = np.empty(48)
coalday = np.empty(48)

for h in range(48):
    gisday[h] = sum(gispower[h::48]) / 365
    modelday[h] = sum(modelpower[h::48]) / 365
    coalday[h] = sum(coalpower[h::48]) / 365

pd.DataFrame(data={'GIS 2009': gisday, 'Model 2009': modelday, 'Coal': coalday}).to_excel('Day.xlsx')


In [ ]:
ep1 = pd.read_csv('EnergyPriceAgg.csv').to_numpy()
print(np.max(ep1,0))
ep1 = np.sum(ep1[:, 2:5],1) / 3
epday = np.empty(48)
for h in range(48):
    epday[h] = sum(ep1[h::48]) / 365
print(sum(epday[16:38]) / 22)

In [38]:
#getting regional info

y = 17520
n = 3*y
solarregout = dict.fromkeys(regions,np.zeros(y)) #every half-hour
solarregcap = dict.fromkeys(regions,0)
windregout = dict.fromkeys(regions,np.zeros(y))
windregcap = dict.fromkeys(regions,0)

#if you want to analyze all GIS sites use this
solarout = solarnames
windout = windnames

#if you want to analyze only 90% model sites use this
# pdat = pd.read_excel('Sites90.xlsx',sheet_name='SitesLong')
# solarout = pdat['Solar'].dropna().to_list()
# windout = pdat['Wind'].dropna().to_list()

#to compare regional load to power
# solarloadout = dict.fromkeys(loadzones,np.zeros(y))
# windloadout = dict.fromkeys(loadzones,np.zeros(y))

#to look at revenue,cost,profit
solarprofit = np.zeros(262)
windprofit = np.zeros(108)

for site in solarout:
    i = solarnames.index(site)
    sitepower = solarpower[:,i]
    cap = solarcaps[i]
    county = solarcounts[i]
    reg = simpregdict[county]
    avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
    solarregout[reg] = solarregout[reg] + avpower #((sitepower[::2] + sitepower[1::2]) / 1) #divide by 2 b/c want avg power (if MWh don't)
    solarregcap[reg] = solarregcap[reg] + cap

    #if you want to compare regional load to power
#     avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
#     cost = solarcost[i]
#     loadreg = weathertoload[reg]
#     solarloadout[loadreg] = solarloadout[loadreg] + avpower
    #if you want to look at revenue,cost,profit
#     rev = np.matmul(energyprice[loadreg],avpower)
#     solarprofit[i] = (rev - cost) #annual
    
for site in windout:
    i = windnames.index(site)
    sitepower = windpower[:,i]
    cap = windcaps[i]
    county = windcounts[i]
    reg = simpregdict[county]
    avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
    windregout[reg] = windregout[reg] + avpower
    windregcap[reg] = windregcap[reg] + cap
    
    #if you want to compare regional load to power
#     avpower = (sitepower[0:y] + sitepower[y:(2*y)] + sitepower[(2*y):(3*y)])/3
#     cost = windcost[i]
#     loadreg = weathertoload[reg]
#     windloadout[loadreg] = windloadout[loadreg] + avpower
    #if you want to look at revenue,cost,profit
#     rev = np.matmul(energyprice[loadreg],avpower)
#     windprofit[i] = (rev - cost) #annual
    
solarregcf = {}
windregcf = {}
regcap = {}
for reg in regions:
    solarregcf[reg] = 2*solarregout[reg] / solarregcap[reg]
    windregcf[reg] = 2*windregout[reg] / windregcap[reg]
    regcap[reg] = [solarregcap[reg],windregcap[reg]]    
#pd.DataFrame(data=regcap).to_csv('RegOut.csv')

solarregsday = {}
windregsday = {}
bothregsday = {}
coalregsday = {}

# mi = 8688
# mf = 10176
# for reg in regions:
#     solarregpower = solarregcf[reg] #getting average capacity factor in region
#     windregpower = windregcf[reg]
#     #coalregpower = coalregscf[reg]
#     solarday = np.empty(48)
#     windday = np.empty(48)
#     coalday = np.empty(24)
#     for h in range(48):
#         solarday[h] = sum(solarregpower[mi+h:mf:48] + solarregpower[mi+h+y:mf+y:48] + solarregpower[mi+h+(2*y):mf+(2*y):48]) / (31*3)
#         windday[h] = sum(windregpower[mi+h:mf:48] + windregpower[mi+h+y:mf+y:48] + windregpower[mi+h+(2*y):mf+(2*y):48]) / (31*3)
#     #for h in range(24):
#         #coalday[h] = sum(coalregpower[h:744:24]) / 31
#     solarregsday[reg] = solarday
#     windregsday[reg] = windday
#     #bothregsday[reg] = solarday + windday
#     #coalregsday[reg] = coalday
    
#pd.DataFrame(data=solarregsday).to_csv('SolarRegionsJul.csv')
#pd.DataFrame(data=windregsday).to_csv('WindRegionsJul.csv')
#pd.DataFrame(data=bothregsday).to_csv('ModelRegions.csv')
#pd.DataFrame(data=coalregsday).to_csv('CoalRegions.csv')


#looking hourly surpluses/losses within each region and grid tranmission 
#simplistic case where any sort of cross regions transmission is just transmission
#- a more realistic case would consider it easier to transmit to neighbours
'''
regsurplus = {}
regloss = {}
gridsurplus = np.zeros(n)
gridloss = np.zeros(n)
for reg in regions:
    dif = solarregout[reg] + windregout[reg] - coalregs[reg]
    surplus = dif.clip(min=0)
    loss = (-1*dif).clip(min=0)
    regsurplus[reg] = surplus
    regloss[reg] = loss
    gridsurplus  = gridsurplus + surplus
    gridloss = gridloss + loss
    
transmission = np.zeros(n)
for h in range(n):
    if gridloss[h] > 0:
        transmission[h] = min(gridloss[h],gridsurplus[h]) #don't send more than u need, can only send what you have
transday = np.zeros(24)        
for h in range(24):
    transday[h] = sum(transmission[h::24]) / (365*3)

pd.DataFrame(data={'Transmission Req': transday}).to_csv('Transmission.csv')
'''

<ipython-input-38-2eb4cdb9dc2a>:70: RuntimeWarning: invalid value encountered in true_divide
  windregcf[reg] = 2*windregout[reg] / windregcap[reg]


"\nregsurplus = {}\nregloss = {}\ngridsurplus = np.zeros(n)\ngridloss = np.zeros(n)\nfor reg in regions:\n    dif = solarregout[reg] + windregout[reg] - coalregs[reg]\n    surplus = dif.clip(min=0)\n    loss = (-1*dif).clip(min=0)\n    regsurplus[reg] = surplus\n    regloss[reg] = loss\n    gridsurplus  = gridsurplus + surplus\n    gridloss = gridloss + loss\n    \ntransmission = np.zeros(n)\nfor h in range(n):\n    if gridloss[h] > 0:\n        transmission[h] = min(gridloss[h],gridsurplus[h]) #don't send more than u need, can only send what you have\ntransday = np.zeros(24)        \nfor h in range(24):\n    transday[h] = sum(transmission[h::24]) / (365*3)\n\npd.DataFrame(data={'Transmission Req': transday}).to_csv('Transmission.csv')\n"

In [53]:
#Monthly regional capacity factor
days = np.array([31,28,31,30,31,30,31,31,30,31,30,31]); halfhours = days*48; hsums = []
for month in range(12):
    hsums.append(sum(halfhours[:month]))
hsums.append(17520)
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

smregs = {}
wmregs = {}
cmregs = {}

targetregs = ['NORTH','COAST','SOUTH','FWEST']
for month in range(12):
    outdic = {}
    for reg in targetregs:
        solaryear = solarregcf[reg]
        windyear = windregcf[reg]
        solarmonth = solaryear[hsums[month]:hsums[month+1]]
        windmonth = windyear[hsums[month]:hsums[month+1]]
        solarday = np.empty(48)
        windday = np.empty(48)
        for hh in range(48):
            solarday[hh] = sum(solarmonth[hh::48]) / days[month]
            windday[hh] = sum(windmonth[hh::48]) / days[month]
        outdic[reg + 'Solar'] = solarday
        outdic[reg + 'Wind'] = windday
    pd.DataFrame(data=outdic).to_csv('AA'+months[month]+'.csv')
        
# pd.DataFrame(data=smregs).to_csv('SolarMonthlyCF.csv')
# pd.DataFrame(data=wmregs).to_csv('WindMonthlyCF.csv')
# pd.DataFrame(data=cmregs).to_csv('CoalMonthlyCF.csv')

In [ ]:
regsums = {}
totals = [0,0,0]
print(regions)
for reg in regions:
    regsums[reg] = [sum(solarregout[reg])/(3*1000),sum(windregout[reg])/(3*1000),sum(coalregs[reg])/1000]
    totals[0] += sum(solarregout[reg])/(3*1000); totals[1] += sum(windregout[reg])/(3*1000)
    totals[2] += sum(coalregs[reg])/1000
revd = 0
powerall = np.zeros(17520)
for reg in loadzones:
    regpower = solarloadout[reg]+windloadout[reg]
    revd += np.matmul(energyprice[reg],regpower)
    print(sum(regpower) / 1000)
    powerall += regpower
rev = np.matmul(energyprice['HB_HUBAVG'],powerall)
revcoal = np.matmul(energyprice['HB_HUBAVG'],coalhours[0:17520])
scale = 10**9
print(sum(coalhours[0:17520]) / 1000)
print(totals)
#print(revd / scale, rev / scale, revcoal / scale)
#print(regsums)
pd.DataFrame(data=regsums).to_csv('RegSumsNew.csv')



In [ ]:
#Testing generalizability to other years
powertest = np.matmul(solarpowertest,sint) + np.matmul(windpowertest,wint)
slacktest = 0 
hourtest = 0
dif = np.transpose(coalhours[0:17520]) - powertest

for i in range(17520):
    if dif[i] > 0:
        slacktest += dif[i]
        hourtest += 1
print(1-(slacktest / (coaltotal/2)))
print(1-(hourtest / 17520))

In [ ]:
#which sites are the most efficient (cost per MW), what range of costs do we have
'''
coalgwh = coaltotal / (3*10**3)
pdat = pd.read_excel('Sites90.xlsx',sheet_name='SitesLong')
solarout = pdat['Solar'].dropna().to_list()
windout = pdat['Wind'].dropna().to_list()
sitesout = pdat['Solar'].dropna().to_list() + pdat['Wind'].to_list()
'''
#2.81 billion

ssum = sum(solarpower) / 3
wsum = sum(windpower) / 3
seff = solarcost / ssum
weff = windcost / wsum
#seff = solarprofit / (10**6)
#weff = windprofit / (10**6)
twine = []

for i in range(windcnum):
    #twine.append((seff[i],ssum[i],solarnames[i]))
    #twine.append((weff[i],wsum[i],windnames[i]))
    twine.append((seff[i],solarcost[i],i,solarnames[i], ssum[i]))
    twine.append((weff[i],windcost[i],i,windnames[i], wsum[i]))
for i in range(windcnum,solarcnum):
    #twine.append((seff[i],ssum[i],solarnames[i]))
    twine.append((seff[i],solarcost[i],i,solarnames[i], ssum[i]))

twine.sort()
#twine.reverse() #need to reverse for falling profitability
order = []
efforder = []
sumorder = []

fcolor = []
totalcost = 0
powerpack = np.zeros(52560)
for i in range(windcnum+solarcnum):
    name = twine[i][3]
    idx = twine[i][2]
    cost = twine[i][1]
    totalcost += cost
    if totalcost > 2.81*(10**9):
        cur = i
        break
    order.append(name)
    efforder.append(twine[i][0])
    #sumorder.append(twine[i][1] / 10**3)
    sumorder.append(twine[i][4] / 10**3)
    if 'Solar:' in name:
        fcolor.append('bisque')
        powerpack += solarpower[:,idx]
    else:
        fcolor.append('lightsteelblue')
        powerpack += windpower[:,idx]
slack = np.maximum(coalhours - powerpack, np.zeros(52560))
slackp = sum(slack) / sum(coalhours)
print(slackp)

coalday = np.empty(48)
cheapday = np.empty(48)
slackday = np.empty(48)
for h in range(48):
    coalday[h] = sum(coalhours[h::48]) / (365*3)
    cheapday[h] = sum(powerpack[h::48]) / (365*3)
    slackday[h] = sum(slack[h::48]) / (365*3)
pd.DataFrame(data={'Coal': coalday,'Cheapest Sites': cheapday, 'Slack': slackday}).to_excel('CheapSites.xlsx')

'''
for site in sitesout:
    i = order.index(site)
    if fcolor[i] == 'bisque':
        fcolor[i] = 'darkorange'
    elif fcolor[i] == 'lightsteelblue':
        fcolor[i] = 'blue'

sp = 0; sc = 0;
wp = 0; wc = 0;

for site in solarout:
    i = solarnames.index(site)
    sp += ssum[i]
    sc += solarcost[i]
for site in windout:
    i = windnames.index(site)
    wp += wsum[i]
    wc += windcost[i]
#print(sp,sc,wp,wc)
print((sc+wc) / (sp+wp))
'''
# popi = []        
# for i in range(0,len(sitesout)-1):
#     n = i+1
#     count = order[i].split(':')[0]
#     sc = sumorder[i]
#     while count == order[n].split(':')[0]:
#         sc += sumorder[n]
#         popi.append(n)
#         n = n+1
#     sumorder[i] = sc
        
# for i in popi:
#     order.pop(i); efforder.pop(i); sumorder.pop(i); fcolor.pop(i);
'''
xs = np.zeros(solarcnum+windcnum)
for i in range(1,solarcnum+windcnum):
    xs[i] = xs[i-1] + sumorder[i-1]
    if abs(efforder[i]) < 0.2:
        efforder[i] += (efforder[i] / (2*abs(efforder[i])))
        print(order[i],efforder[i])

srange = [min(seff),max(seff)]
wrange = [min(weff),max(weff)]
print('Solar Price Per MWh Range: %s' % srange)
print('Wind Price Per MWh Range: %s' % wrange)
'''
# for i in range(windcnum):
#     sorder.append(stwine[i][1])
#     worder.append(wtwine[i][1])
# for i in range(windcnum,solarcnum):
#     sorder.append(stwine[i][1])

# sorder = list(dict.fromkeys(sorder)) #remove repeats
# worder = list(dict.fromkeys(worder))
# sorder = [count + ' Solar' for count in sorder]
# worder = [count + ' Wind' for count in worder]

#basically want to see whether its taking greedy 'best' sites or not
# sg = defaultdict(float)
# wg = defaultdict(float)


# for count in solarout:
#     [x,y] = count.split(':')
#     i = sorder.index(x)
#     sg[i+1] += float(y)
# for count in windout:
#     [x,y] = count.split(':')
#     i = worder.index(x)
#     wg[i+1] += float(y)

# sg = dict(sorted(sg.items()))
# wg = dict(sorted(wg.items()))

# pd.DataFrame(data=[sg,wg]).to_csv('Eff90Full.csv')

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter
import math

fig,ax = plt.subplots(1, figsize=(15,3))
#plt.grid(zorder=0); ax.set_axisbelow(True)
plt.bar(x=xs, height=efforder, width=sumorder, align='edge', color=fcolor)
plt.axvline(x=coalgwh,linestyle='--',color='black'); plt.text(coalgwh+1000,40,'Coal GWh')
plt.xlabel('Annual Energy Supplied (GWh)'); plt.ylabel('Levelized Cost ($ / MWh)');
ax.set_xlim((0,xs[-1]+5000)); 
ax.xaxis.set_major_locator(MultipleLocator(25000)); #ax.xaxis.set_minor_locator(MultipleLocator(25000))
ax.get_xaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))

ax.yaxis.set_minor_locator(MultipleLocator(5))
ax.spines['bottom'].set_color('gray'); ax.spines['top'].set_color('white');
ax.spines['right'].set_color('white'); ax.spines['left'].set_color('gray');

lcolors = {'Wind Used':'blue', 'Wind Unused':'lightsteelblue','Solar Used':'darkorange', 'Solar Unused':'bisque'}   
lecolors = {'Wind Used':'blue', 'Wind Unused':'lightsteelblue','Solar Used':'darkorange', 'Solar Unused':'bisque'} 
labels = list(lcolors.keys())
handles = [plt.Rectangle((0,0),1,1, facecolor=lcolors[label], edgecolor=lecolors[label]) for label in labels]
plt.legend(handles, labels)
plt.tight_layout()

#plt.savefig('Site_Selection_Bar_AnnualProfit')

In [ ]:
#Quartiles
csv = 'Power90.csv'
pdat = pd.read_csv(csv)
allpower = 2*(pdat['Solar'].to_numpy() + pdat['Wind'].to_numpy())
print(allpower.shape)
n = 17520

year1 = allpower[0:n].flatten()
year2 = allpower[n:(2*n)].flatten()
year3 = allpower[(2*n):(3*n)].flatten()

day1 = np.empty(48)
day2 = np.empty(48)
day3 = np.empty(48)
jan1 = np.empty(48)
jan2 = np.empty(48)
jan3 = np.empty(48)
jul1 = np.empty(48)
jul2 = np.empty(48)
jul3 = np.empty(48)
#aprall = np.empty(48)
#octall = np.empty(48)

for i in range(48):
    day = np.sort(allpower[i::48])   #(year1[i::48])
    day1[i] = (day[273]) #1st quratile ie median of first half (day[90] + day[91])/2
    day2[i] = day[547] #2nd quartile ie median (day[182])
    day3[i] = day[821]# (day[273] + day[274]) / 2
    jan = np.sort(np.hstack((year1[i:1488:48],year2[i:1488:48],year3[i:1488:48])))   #(year1[i:1488:48])
    jan1[i] = (jan[22] + jan[23]) / 2 #jan[7]
    jan2[i] = jan[46]#jan[15]
    jan3[i] = (jan[69] + jan[70]) / 2 #jan[23]
    jul = np.sort(np.hstack((year1[8688+i:10176:48],year2[8688+i:10176:48],year3[8688+i:10176:48])))   #(year1[8688+i:10176:48])
    jul1[i] = (jul[22] + jul[23]) / 2 #jul[7]
    jul2[i] = jul[46] #jul[15]
    jul3[i] = (jul[69] + jul[70]) / 2 #jul[23]
    #aprall[i] = (sum(year1[4320+i:5760:48]) + sum(year2[4320+i:5760:48]) + sum(year3[4320+i:5760:48])) / (3*30)
    #octall[i] = (sum(year1[13104+i:14592:48]) + sum(year2[13104+i:14592:48]) + sum(year3[13104+i:14592:48])) / (3*31)

#years = {'2009 Year': year1, '2010 Year': year2, '2011 Year': year3}
#alld = {'2009 Day': day1, '2010 Day': day2, '2011 Day': day3}
alld = {'Year Q1': day1, 'Year Q2': day2, 'Year Q3': day3}
alld['Jan Q1'] = jan1
alld['Jan Q2'] = jan2
alld['Jan Q3'] = jan3
alld['Jul Q1'] = jul1 
alld['Jul Q2'] = jul2
alld['Jul Q3'] = jul3
#aproct = {'April': aprall, 'October': octall}

pd.DataFrame(data=alld).to_excel('Model90Qs.xlsx')

In [63]:
#fuel mix info

#fuel = 'Fuel2011.xls' 
#months = ['Jan11','Feb11','Mar11','Apr11','May11','Jun11','Jul11','Aug11','Sep11','Oct11','Nov11','Dec11']
fuel = 'Fuel2019.xlsx'
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
xls = pd.ExcelFile(fuel)

biomass = np.empty(0)
coal = np.empty(0)
gas = np.empty(0)
gascc = np.empty(0)
hydro = np.empty(0)
nuclear = np.empty(0)
other = np.empty(0)
solar = np.empty(0)
wind = np.empty(0)



for i in range(12):
    dat = pd.read_excel(xls, months[i])
    
    biomassyear = dat.loc[dat['Fuel']=="Biomass"]
    biomassyear = biomassyear[biomassyear.columns[4::2]].to_numpy() + biomassyear[biomassyear.columns[5::2]].to_numpy()
    biomassyear = np.reshape(biomassyear,np.prod(biomassyear.shape))
    biomass = np.concatenate((biomass,biomassyear))
    
    coalyear = dat.loc[dat['Fuel']=="Coal"]
    coalyear = coalyear[coalyear.columns[4::2]].to_numpy() + coalyear[coalyear.columns[5::2]].to_numpy()
    coalyear = np.reshape(coalyear,np.prod(coalyear.shape))
    coal = np.concatenate((coal,coalyear))
    
    gasyear = dat.loc[dat['Fuel']=="Gas"]
    gasyear = gasyear[gasyear.columns[4::2]].to_numpy() + gasyear[gasyear.columns[5::2]].to_numpy()
    gasyear = np.reshape(gasyear,np.prod(gasyear.shape))
    gas = np.concatenate((gas,gasyear))

    gasccyear = dat.loc[dat['Fuel']=="Gas-CC"]
    gasccyear = gasccyear[gasccyear.columns[4::2]].to_numpy() + gasccyear[gasccyear.columns[5::2]].to_numpy()
    gasccyear = np.reshape(gasccyear,np.prod(gasccyear.shape))
    gascc = np.concatenate((gascc,gasccyear))

    hydroyear = dat.loc[dat['Fuel']=="Hydro"]
    hydroyear = hydroyear[hydroyear.columns[4::2]].to_numpy() + hydroyear[hydroyear.columns[5::2]].to_numpy()
    hydroyear = np.reshape(hydroyear,np.prod(hydroyear.shape))
    hydro = np.concatenate((hydro,hydroyear))
    
    nuclearyear = dat.loc[dat['Fuel']=="Nuclear"]
    nuclearyear = nuclearyear[nuclearyear.columns[4::2]].to_numpy() + nuclearyear[nuclearyear.columns[5::2]].to_numpy()
    nuclearyear = np.reshape(nuclearyear,np.prod(nuclearyear.shape))
    nuclear = np.concatenate((nuclear,nuclearyear))
    
    otheryear = dat.loc[dat['Fuel']=="Other"]
    otheryear = otheryear[otheryear.columns[4::2]].to_numpy() + otheryear[otheryear.columns[5::2]].to_numpy()
    otheryear = np.reshape(otheryear,np.prod(otheryear.shape))
    other = np.concatenate((other,otheryear))

    solaryear = dat.loc[dat['Fuel']=="Solar"]
    solaryear = solaryear[solaryear.columns[4::2]].to_numpy() + solaryear[solaryear.columns[5::2]].to_numpy()
    solaryear = np.reshape(solaryear,np.prod(solaryear.shape))
    solar = np.concatenate((solar,solaryear))

    windyear = dat.loc[dat['Fuel']=="Wind"]
    windyear = windyear[windyear.columns[4::2]].to_numpy() + windyear[windyear.columns[5::2]].to_numpy()
    windyear = np.reshape(windyear,np.prod(windyear.shape))
    wind = np.concatenate((wind,windyear))
    
biomass = np.nan_to_num(biomass)
coal = np.nan_to_num(coal) # / (18774/2) # (14297/2)
gas = np.nan_to_num(gas) # / (47259/2)
gascc = np.nan_to_num(gascc)
hydro = np.nan_to_num(hydro)
nuclear = np.nan_to_num(nuclear)
other = np.nan_to_num(other)
solar = np.nan_to_num(solar)
wind = np.nan_to_num(wind) # / (9452/2)
solar = np.nan_to_num(solar)
#solar = pd.read_excel('SolarProfile2011.xlsx',usecols=['Total']).to_numpy().flatten()
#solar = solar / max(solar)
#oldrenewables = wind+other+hydro
#total = coal+gas+nuclear+oldrenewables
oldrenewables = solar+wind+other+biomass+hydro
total = coal+gas+gascc+nuclear+oldrenewables
total3 = np.hstack((total,total,total))

pdat = pd.read_csv('Power90.csv')
solarp = pdat['Solar'].to_numpy()
windp = pdat['Wind'].to_numpy()
'''
solarpower = np.empty(17520)
windpower = np.empty(17520)
for i in range(17520):
    solarpower[i] = sum(solarp[i::17520])/3
    windpower[i] = sum(windp[i::17520])/3
    
pdat = pd.read_excel('Sites90.xlsx',sheet_name='SitesLong')
solarout = pdat['Solar'].dropna().to_list()
windout = pdat['Wind'].dropna().to_list()
sitesout = pdat['Solar'].dropna().to_list() + pdat['Wind'].to_list()

solarmodel = np.zeros(52560)
windmodel = np.zeros(52560)
sc = 0; wc = 0;

for site in solarout:
    i = solarnames.index(site)
    solarmodel += solarpower[:,i]
    cap = float(site.split(': ')[1])
    sc += cap
for site in windout:
    i = windnames.index(site)
    windmodel += windpower[:,i]
    cap = float(site.split(': ')[1])
    wc += cap

totalmodel = 2*(solarmodel + windmodel) / (sc+wc)
solarmodel = 2*solarmodel / sc
windmodel = 2*windmodel / wc

gas3 = 2*(gas + gascc) / (19310 + 35564)
gas3 = np.hstack((gas3,gas3,gas3))

solarmodelh = (solarmodel[::2] + solarmodel[1::2]) / 2
windmodelh = (windmodel[::2] + windmodel[1::2]) / 2
windh = (wind[::2] + wind[1::2]) / 2
coalh = (coal[::2] + coal[1::2]) / 2
gash = (gas[::2] + gas[1::2]) / 2
print(coalh.shape)
'''

load2009 = pd.read_excel('2009_ERCOT_Hourly_Load_Data.xls',usecols=['ERCOT']).to_numpy().flatten()
load2009 = np.repeat(load2009, 2)
load2010 = pd.read_excel('2010_ERCOT_Hourly_Load_Data.xls',usecols=['ERCOT']).to_numpy().flatten()
load2010 = np.repeat(load2010, 2)
load2011 = pd.read_excel('Native_Load_2011.xls',usecols=['ERCOT']).to_numpy().flatten()
load2011 = np.repeat(load2011, 2)
load = pd.read_excel('Native_Load_2019.xlsx',usecols=['ERCOT']).to_numpy().flatten()
load2019 = np.repeat(load, 2)

y = 17520
gas2019 = gas + gascc
coal2019 = coal
outdic = {'Load 2009': load2009,'Load 2010': load2010,'Load 2011': load2011, 'Load 2019': load2019, 'Solar 2009': 2*solarp[0:y],
          'Solar 2010': 2*solarp[y:(2*y)], 'Solar 2011': 2*solarp[(2*y):(3*y)], 'Wind 2009': 2*windp[0:y],
          'Wind 2010': 2*windp[y:(2*y)], 'Wind 2011': 2*windp[(2*y):(3*y)], 'Coal 2019': 2*coal2019, 'Gas 2019': 2*gas2019}
# pd.DataFrame(data = outdic).to_csv('Output_vs_Load.csv')



load3 = np.hstack((load,load,load))

#oldrenewablesh = oldrenewables[::2] + oldrenewables[1::2]
coalh = coal[::2] + coal[1::2]
solarp = solarp[::2] + solarp[1::2]
windp= windp[::2] + windp[1::2]

#oldrenew3 = np.hstack((oldrenewablesh,oldrenewablesh,oldrenewablesh))
coal3 = np.hstack((coalh,coalh,coalh))
old = load3 - coal3
new = load3 - (solarp+windp)

In [ ]:
#slack (based off fuel mix data)
dif = coal3 - (solarp+windp)
slack = np.empty(52560)
for i in range(52560):
    slack[i] = max(0,dif[i])

difday = np.empty(48)
slackday = np.empty(48)
s1 = np.empty(48)
s2 = np.empty(48)
s3 = np.empty(48)

#slack quartilers
for i in range(48):
    sh = np.sort(slack[i::48])
    s1[i] = 2*sh[273]
    s2[i] = 2*sh[547]
    s3[i] = 2*sh[821]
    difday[i] = sum(dif[i::48]) / (365*3)
    slackday[i] = sum(slack[i::48]) / (365*3)
#slack quartiles
pd.DataFrame(data={'Slack Q1':s1, 'Slack Q2': s2, 'Slack Q3': s3}).to_csv('SlackQs.csv')
#average slack day and difference day
pd.DataFrame(data={'Dif':difday, 'Slack': slackday}).to_excel('SlackOut.xlsx')

In [72]:
#Required Ramping of other energy sources to satisfy load (based off fuel mix and model data)
yh = 8760
oldavg = sum(old) / (yh*3)
newavg = sum(new)/ (yh*3)
oldpeak = max(old)
newpeak = max(new)
oldstd = np.std(old)
newstd = np.std(new)
print(oldpeak/oldavg,oldstd,newpeak/newavg,newstd)

#monthly comparison of old and new required ramping
#can easily be done for full year if mi = 0, mf = yh, and days[month] is set to 365
outday = {}
for month in range(12):
    oldday = np.empty(24)
    newday = np.empty(24)
    mi = hsums[month] // 2 #divide by 2 because half-hours, no rounding errors in division (all will be integers)
    mf = hsums[month+1] // 2
    for h in range(24):
        oldday[h] = sum(old[mi+h:mf:24] + old[mi+h+yh:mf+yh:24] + old[mi+h+(2*yh):mf+(2*yh):24]) / (days[month]*3)
        newday[h] = sum(new[mi+h:mf:24] + new[mi+h+yh:mf+yh:24] + new[mi+h+(2*yh):mf+(2*yh):24]) / (days[month]*3)
        outday['Old' + months[month]] = oldday
        outday['New' + months[month]] = newday

#top level analysis
olddist = np.flip(np.sort(old / oldavg))
newdist = np.flip(np.sort(new / newavg))
outlong = {'Old Req':old, 'New Req': new}
outdist = {'Old Req Dist':olddist, 'New Req Dist': newdist}

#pd.DataFrame(data=outlong).to_excel('RequiredRamping.xlsx')
pd.DataFrame(data=outday).to_excel('RequiredRampingSupplement.xlsx')
#pd.DataFrame(data=outdist).to_excel('RequiredRamping.xlsx')

#plt.plot(new, color='blue')
#plt.plot(old, color='orange')

1.779243436500902 8035.444593707889 1.9653871321836818 10102.21130075584


In [ ]:
#fuel mix graph
bh = np.empty(48); ch = np.empty(48); gh = np.empty(48); gcch = np.empty(48); hh = np.empty(48); nh = np.empty(48);
oh = np.empty(48); sh = np.empty(48); wh = np.empty(48); sph = np.empty(48); wph = np.empty(48);

mi = 8688 #0
mf = 10176 #1488
dn = 31

for h in range(48):
    bh[h] = (2*sum(biomass[mi+h:mf:48])) / dn
    ch[h] = (2*sum(coal[mi+h:mf:48])) / dn
    gh[h] = (2*sum(gas[mi+h:mf:48])) / dn
    gcch[h] = (2*sum(gascc[mi+h:mf:48])) / dn
    hh[h] = (2*sum(hydro[mi+h:mf:48])) / dn
    nh[h] = (2*sum(nuclear[mi+h:mf:48])) / dn
    oh[h] = (2*sum(other[mi+h:mf:48])) / dn
    sh[h] = (2*sum(solar[mi+h:mf:48])) / dn
    wh[h] = (2*sum(wind[mi+h:mf:48])) / dn
    sph[h] = (2*sum(solarpower[mi+h:mf:48])) / dn
    wph[h] = (2*sum(windpower[mi+h:mf:48])) / dn
    
#formatting to make that graph I want (basically need to put things on top of one another)
total = nh+hh+bh+oh+sh+wh+gh+gcch+ch
    
outdic = {'Nuclear':nh, 'Hydrothermal/Biomass/Other':hh+bh+oh, 'Solar':sh, 'New Solar':sph, 'Wind':wh, 'New Wind':wph, 'Gas-CC':gcch, 'Gas':gh, 'Coal':ch, 'Total ERCOT Generation':total}

pd.DataFrame(data=outdic).to_excel('FuelHoursJul.xlsx') 

In [ ]:
days = np.array([31,28,31,30,31,30,31,31,30,31,30,31]); halfhours = days*48; hsums = []
months = ['January','February','March','April','May','June','July','August','September','October','November','December']
for m in range(12):
    hsums.append(sum(halfhours[:m]))
hsums.append(17520)

In [ ]:
#Slack by Month
slack = pd.read_csv('Slack90.csv',usecols=['Slack']).to_numpy().flatten()

months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
daysog = [31,28,31,30,31,30,31,31,30,31,30,31]
daysleap = [31,29,31,30,31,30,31,31,30,31,30,31]
t = 17520
yrhours = [t,t,t] #for leap this would be t + 48
baseyear = 2009

ms = np.zeros(12)
cms = np.zeros(12)
ys = np.zeros(3)

for halfhour in range(len(slack)):
    oghh = halfhour
    yi = 0
    while halfhour >= yrhours[yi]:
        halfhour -= yrhours[yi]
        yi += 1
    
    year = baseyear + yi
    
    if (year % 4) == 0:
        days = daysleap
    else:
        days = daysog

    day = (halfhour // 48) + 1
    mi = 0
    while day > days[mi]:
        day -= days[mi]
        mi += 1  
    
    month = months[mi]
    hour = (halfhour % 48) / 2
    ys[yi] += slack[oghh]
    ms[mi] += slack[oghh]
    
for halfhour in range(t):
    oghh = halfhour

    days = daysog
    day = (halfhour // 48) + 1
    mi = 0
    while day > days[mi]:
        day -= days[mi]
        mi += 1  
    
    month = months[mi]
    cms[mi] += coalhours[oghh]

pd.DataFrame(data={'Slack':ms}).to_csv('SlackMonths.csv')   
pd.DataFrame(data={'Slack':ys}).to_csv('SlackYears.csv')
pd.DataFrame(data={'Coal':cms}).to_csv('CoalMonths.csv')

In [ ]:
#Study Phase info
gisreport = 'Ercot_GIS_Report_June.xlsx'
pdat = pd.read_excel(gisreport, sheet_name='Project Details', usecols=['GINR Study Phase','County','Fuel','Capacity (MW)'])

sphase = solarp['GINR Study Phase'].to_list()
wphase = windp['GINR Study Phase'].to_list()

spd = defaultdict(int)
wpd = defaultdict(int)

for site in solarout:
    i = solarnames.index(site)
    phase = sphase[i]
    #[name,cap] = site.split(' Solar: ')
    cap = solarcaps[i]
    spd[phase] += float(cap)
for site in windout:
    i = windnames.index(site)
    phase = wphase[i]
    #[name,cap] = site.split(' Wind: ')
    cap = windcaps[i]
    wpd[phase] += float(cap)
pd.DataFrame(data=[spd,wpd]).to_excel('StudyPhaseOut.xlsx')


In [ ]:
coal = 'Coal2019.csv'
coalhours = pd.read_csv(coal,usecols=['Coal']).to_numpy().flatten()
ep = energyprice['HB_HUBAVG'].to_numpy().flatten()
r = np.matmul(coalhours,ep)
print(r*0.9 / (10**9))
#coalhours = np.hstack((coalhours,coalhours,coalhours))
coalcf = coalhours / max(coalhours)

# pdat = pd.read_csv('Power90.csv')
# allpower = pdat['Solar'].to_numpy() + pdat['Wind'].to_numpy()

# #print((2.5*10**9)/(sum(coalhours)*0.7))
# dif = coalhours - allpower
# hc = 0
# for i in range(52560):
#     if dif[i] > 0:
#         hc += 1
# print(hc/52560)
# coalday = np.empty(48)
# for h in range(48):
#     pass
#    coalday[h] = sum(coalcf[8688+h:10176:48]) / 31

#pd.DataFrame(data={'Coal':coalday}).to_csv('CoalJul.csv')